- In this notebook, we convert the original GPT architecture into a Llama 2 model step by step (note the GPT and GPT-2 share the same architecture)
- Why not Llama 1 or Llama 3?
   - The Llama 1 architecture is similar to Llama 2, except that Llama 2 has a larger context window (which is nice); the Llama 1 weights are not readily available and have more usage restrictions, so it makes more sense to focus on Llama 2
   - Regarding Llama 3, I will share a separate notebook to convert Llama 2 to Llama 3 (there are only a few small additional changes)
- The explanations are purposefully kept minimal in this notebook not to bloat it unnecessarily and focus on the main code
- For more information, please see the Llama 2 paper: [Llama 2: Open Foundation and Fine-Tuned Chat Models (2023)](https://arxiv.org/abs/2307.09288)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/gpt2-to-llama2-llama3.webp?1">

In [3]:
# !pip install sentencepiece

In [4]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",  # to download pretrained weights
    "sentencepiece",    # to implement the tokenizer
    "torch",            # to implement the model
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

huggingface_hub version: 0.27.0
sentencepiece version: 0.2.0
torch version: 2.5.1


# 1. Convert the GPT model implementation step by step

- In this section, we go through the GPT model code from [chapter 4](../../ch04/01_main-chapter-code/ch04.ipynb) and modify it step by step to implement the Llama 2 architecture
- Later, we load the original Llama 2 weights shared by Meta AI

&nbsp;
## 1.1 Replace LayerNorm with RMSNorm layer

- First, we replace LayerNorm by Root Mean Square Layer Normalization (RMSNorm)
- LayerNorm normalizes inputs using mean and variance, while RMSNorm uses only the root mean square, which improves computational efficiency
- The RMSNorm operation is as follows, where $x$ is the input $\gamma$ is a trainable parameter (vector), and $\epsilon$ is a small constant to avoid zero-division errors:

$$y_i = \frac{x_i}{\text{RMS}(x)} \gamma_i, \quad \text{where} \quad \text{RMS}(x) = \sqrt{\epsilon + \frac{1}{n} \sum x_i^2}$$

- For more details, please see the paper [Root Mean Square Layer Normalization (2019)](https://arxiv.org/abs/1910.07467)

In [5]:
import torch
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.emb_dim = emb_dim
        self.weight = nn.Parameter(torch.ones(emb_dim)).float()

    def forward(self, x):
        means = x.pow(2).mean(dim=-1, keepdim=True)
        x_normed = x * torch.rsqrt(means + self.eps)
        return (x_normed * self.weight).to(dtype=x.dtype)

In [6]:
torch.manual_seed(123)

example_batch = torch.randn(2, 3, 4)

rms_norm = RMSNorm(emb_dim=example_batch.shape[-1])
rmsnorm_pytorch = torch.nn.RMSNorm(example_batch.shape[-1], eps=1e-5)

assert torch.allclose(rms_norm(example_batch), rmsnorm_pytorch(example_batch))

&nbsp;
## 1.2 Replace GELU with SiLU activation

- Llama uses the SiLU activation function (instead of GELU), which is also known as the Swish function:

$$
\text{silu}(x) = x \cdot \sigma(x), \quad \text{where} \quad \sigma(x) \text{ is the logistic sigmoid.}
$$

- For more information, see the SiLU paper: [Sigmoid-Weighted Linear Units for Neural Network Function Approximation in Reinforcement Learning (2017)](https://arxiv.org/abs/1702.03118)


In [7]:
class SiLU(nn.Module):
    def __init__(self):
        super(SiLU, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)

In [9]:
example_batch.shape

torch.Size([2, 3, 4])

In [8]:
silu = SiLU()

assert torch.allclose(silu(example_batch), torch.nn.functional.silu(example_batch))

&nbsp;
## 1.3 Update the FeedForward module

- In fact, Llama uses a "Gates Linear Unit" (GLU) variant of SiLU called SwiGLU, which essentially results in a slightly differently structured `FeedForward` module
- SwiGLU uses a gating mechanism in the feedforward layer, with the formula:

$$\text{SwiGLU}(x) = \text{SiLU}(\text{Linear}_1(x)) * (\text{Linear}_2(x))$$

- Here, $\text{Linear}_1$ and $\text{Linear}_2$ are two linear layers, and $*$ denotes element-wise multiplication
- The third linear layer, $\text{Linear}_3$, is applied after this gated activation

- For more information, see SwiGLU paper: [GLU Variants Improve Transformer (2020)](https://arxiv.org/abs/2002.05202)


In [10]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)
        self.silu = SiLU()

    def forward(self, x):
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)
        x = self.silu(x_fc1) * x_fc2
        return self.fc3(x)

- Note that we also added a `dtype=cfg["dtype"]` setting above, which will allow us to load the model directly in lower precision formats later to reduce memory usage (versus instantiating it in the original 32-bit precision format and then converting it)
- We also set `bias=False` since Llama doesn't use any bias units

&nbsp;
## 1.4 Implement RoPE

Rotary Position Embeddings (RoPE) enhance positional encoding in transformers by introducing relative position information through a rotation matrix. Here's a conceptual and mathematical breakdown of how it works:

---

### **Conceptual Explanation**
1. **Problem**: Traditional positional embeddings (absolute or relative) are often additive, combined with token embeddings, and may not efficiently handle relative positions, especially for long sequences.

2. **RoPE Approach**:
   - Encodes positional information via rotations in a multi-dimensional space.
   - Applies a rotation matrix directly to query and key vectors in the self-attention mechanism.
   - Ensures that relative positional relationships between tokens influence the attention computation.

3. **Benefits**:
   - Maintains relative positional dependency efficiently.
   - Exhibits long-term decay, reducing the attention weight for distant tokens naturally.
   - Flexible for varying sequence lengths and compatible with linear attention mechanisms.

---

### **Mathematical Formulation**
1. **Positional Encoding as Rotation**:
   For a token at position $ m $, the position-encoded query and key are computed as:
   
   $$q_m = W_q x_m \cdot e^{i m \theta}, \quad k_n = W_k x_n \cdot e^{i n \theta}$$

   Here:
   - $ W_q, W_k $: Linear transformation weights for queries and keys.
   - $ x_m, x_n $: Embeddings of tokens at positions $ m $ and $ n $.
   - $ \theta $: Rotation angle per dimension, often parameterized based on $ \theta_i = 10000^{-2i/d} $, where $ d $ is the embedding size.

2. **Relative Position Encoding**:
   The inner product between a query and a key incorporates relative positions:
   $$
   q_m^\top k_n = (W_q x_m)^\top (W_k x_n) \cdot e^{i (m - n) \theta}
   $$
   This introduces the relative position $ m - n $ as a rotation in the complex plane.

3. **Generalized Rotation Matrix**:
   For $ d $-dimensional embeddings with even $ d $, the rotation matrix $ R_{\Theta, m} $ is block-diagonal:
   $$
   R_{\Theta, m} = 
   \begin{bmatrix}
   \cos(m\theta_1) & -\sin(m\theta_1) & 0 & \cdots & 0 \\
   \sin(m\theta_1) & \cos(m\theta_1) & 0 & \cdots & 0 \\
   0 & 0 & \cos(m\theta_2) & \cdots & 0 \\
   0 & 0 & \sin(m\theta_2) & \cdots & 0 \\
   \vdots & \vdots & \vdots & \ddots & \vdots \\
   0 & 0 & 0 & \cdots & \cos(m\theta_{d/2}) \\
   0 & 0 & 0 & \cdots & \sin(m\theta_{d/2}) \\
   \end{bmatrix}
   $$
   Queries and keys are transformed as:
   $$
   q_m = R_{\Theta, m} W_q x_m, \quad k_n = R_{\Theta, n} W_k x_n
   $$

4. **Self-Attention with RoPE**:
   The self-attention mechanism becomes:
   $$
   q_m^\top k_n = (W_q x_m)^\top R_{\Theta, n-m} W_k x_n
   $$
   Here, $ R_{\Theta, n-m} = R_{\Theta, m}^\top R_{\Theta, n} $ encodes relative positions.

5. **Long-Term Decay**:
   The rotation parameter $ \theta_i = 10000^{-2i/d} $ ensures that contributions from distant tokens decay exponentially, aligning with natural language dependencies.

---

### **Key Properties**
- **Multiplicative Encoding**: Unlike additive embeddings, RoPE's encoding through rotation maintains the magnitude of the embeddings.
- **Compatibility**: Supports linear attention mechanisms, as the rotation does not alter vector norms.
- **Efficiency**: Computational overhead is minimal due to the sparsity of the rotation matrix.

This approach is elegant because it embeds relative position information directly into the attention mechanism, leveraging the geometric properties of rotations, and aligns well with the self-attention framework.

In [11]:
# Dummy embeddings: Batch size 2, Sequence length 4, Embedding dimension 6
B, L, d = 2, 4, 6
dummy_embeddings = torch.randn(B, L, d)

In [12]:
dummy_embeddings.shape

torch.Size([2, 4, 6])

In [13]:
embeddings = dummy_embeddings
seq_len = L
dim = d

In [14]:
# Generate position indices (0 to seq_len-1)
positions = torch.arange(seq_len, dtype=torch.float32).unsqueeze(1)  # Shape: (L, 1)

In [15]:
positions

tensor([[0.],
        [1.],
        [2.],
        [3.]])

In [16]:
# Compute rotation angles for each dimension
dim_indices = torch.arange(dim // 2, dtype=torch.float32)  # Shape: (d/2,)
theta = 10000 ** (-2 * dim_indices / dim)  # Shape: (d/2,)

In [17]:
theta

tensor([1.0000, 0.0464, 0.0022])

In [19]:
# Compute the angles: pos * theta
angles = positions * theta  # Shape: (L, d/2)
angles

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 4.6416e-02, 2.1544e-03],
        [2.0000e+00, 9.2832e-02, 4.3089e-03],
        [3.0000e+00, 1.3925e-01, 6.4633e-03]])

In [20]:
# Compute cos and sin of the angles
cos_angles = torch.cos(angles)  # Shape: (L, d/2)
sin_angles = torch.sin(angles)  # Shape: (L, d/2)

In [21]:
cos_angles

tensor([[ 1.0000,  1.0000,  1.0000],
        [ 0.5403,  0.9989,  1.0000],
        [-0.4161,  0.9957,  1.0000],
        [-0.9900,  0.9903,  1.0000]])

In [22]:
sin_angles

tensor([[0.0000, 0.0000, 0.0000],
        [0.8415, 0.0464, 0.0022],
        [0.9093, 0.0927, 0.0043],
        [0.1411, 0.1388, 0.0065]])

In [23]:
# Expand cos and sin to interleave dimensions
cos_angles = torch.stack([cos_angles, cos_angles], dim=-1).reshape(seq_len, dim)  # Shape: (L, d)
sin_angles = torch.stack([sin_angles, sin_angles], dim=-1).reshape(seq_len, dim)  # Shape: (L, d)


In [24]:
cos_angles

tensor([[ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000],
        [ 0.5403,  0.5403,  0.9989,  0.9989,  1.0000,  1.0000],
        [-0.4161, -0.4161,  0.9957,  0.9957,  1.0000,  1.0000],
        [-0.9900, -0.9900,  0.9903,  0.9903,  1.0000,  1.0000]])

In [25]:
sin_angles

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8415, 0.8415, 0.0464, 0.0464, 0.0022, 0.0022],
        [0.9093, 0.9093, 0.0927, 0.0927, 0.0043, 0.0043],
        [0.1411, 0.1411, 0.1388, 0.1388, 0.0065, 0.0065]])

In [26]:
# Apply rotations to embeddings
embeddings_cos = embeddings * cos_angles  # Shape: (B, L, d)
embeddings_sin = embeddings.roll(shifts=-1, dims=-1) * sin_angles  # Rotate embeddings by 1 along dim
rotated_embeddings = embeddings_cos + embeddings_sin

In [28]:
embeddings.shape

torch.Size([2, 4, 6])

In [29]:
cos_angles.shape

torch.Size([4, 6])

In [27]:
embeddings_cos

tensor([[[ 0.9624,  0.2492, -0.4845, -2.0929, -0.8199, -0.4210],
         [-0.5198,  0.6929, -0.3427, -0.6814, -0.9887, -1.7018],
         [ 0.3120,  0.4696,  0.4117,  0.2879, -0.2044, -2.2685],
         [ 0.9042,  0.4162,  1.2984, -0.2178,  0.2189,  0.2045]],

        [[ 0.6177, -0.2876,  0.8218,  0.1512,  0.1036, -2.1996],
         [-0.0478, -0.3032, -0.0635, -0.4482, -1.2785, -1.1440],
         [-0.1014,  0.0236,  0.3768,  1.6790, -0.5581, -0.0517],
         [ 0.8992, -0.3472,  1.5284,  0.1392,  1.0617, -0.9929]]])

In [31]:
torch.set_printoptions(sci_mode=False)

In [32]:
embeddings_sin

tensor([[[     0.0000,     -0.0000,     -0.0000,     -0.0000,     -0.0000,
               0.0000],
         [     1.0792,     -0.2887,     -0.0317,     -0.0459,     -0.0037,
              -0.0021],
         [    -1.0262,      0.3760,      0.0268,     -0.0190,     -0.0098,
              -0.0032],
         [    -0.0593,      0.1850,     -0.0305,      0.0304,      0.0013,
              -0.0059]],

        [[    -0.0000,      0.0000,      0.0000,      0.0000,     -0.0000,
               0.0000],
         [    -0.4722,     -0.0535,     -0.0208,     -0.0593,     -0.0025,
              -0.0002],
         [    -0.0516,      0.3441,      0.1563,     -0.0517,     -0.0002,
               0.0010],
         [     0.0495,      0.2178,      0.0195,      0.1474,     -0.0064,
              -0.0059]]])

In [33]:
rotated_embeddings

tensor([[[ 0.9624,  0.2492, -0.4845, -2.0929, -0.8199, -0.4210],
         [ 0.5594,  0.4043, -0.3743, -0.7273, -0.9924, -1.7039],
         [-0.7141,  0.8456,  0.4385,  0.2690, -0.2142, -2.2717],
         [ 0.8449,  0.6012,  1.2678, -0.1874,  0.2203,  0.1986]],

        [[ 0.6177, -0.2876,  0.8218,  0.1512,  0.1036, -2.1996],
         [-0.5200, -0.3567, -0.0843, -0.5075, -1.2809, -1.1442],
         [-0.1530,  0.3677,  0.5331,  1.6273, -0.5583, -0.0506],
         [ 0.9487, -0.1294,  1.5479,  0.2866,  1.0553, -0.9988]]])

## Back to RoPE

In [34]:
def precompute_rope_params(head_dim, theta_base=10_000, context_length=4096):
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # Compute the inverse frequencies
    inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    # Generate position indices
    positions = torch.arange(context_length)

    # Compute the angles
    angles = positions[:, None] * inv_freq[None, :]  # Shape: (context_length, head_dim // 2)

    # Expand angles to match the head_dim
    angles = torch.cat([angles, angles], dim=1)  # Shape: (context_length, head_dim)

    # Precompute sine and cosine
    cos = torch.cos(angles)
    sin = torch.sin(angles)

    return cos, sin

def compute_rope(x, cos, sin):
    # x: (batch_size, num_heads, seq_len, head_dim)
    batch_size, num_heads, seq_len, head_dim = x.shape
    assert head_dim % 2 == 0, "Head dimension must be even"

    # Split x into first half and second half
    x1 = x[..., : head_dim // 2]  # First half
    x2 = x[..., head_dim // 2 :]  # Second half

    # Adjust sin and cos shapes
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, seq_len, head_dim)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # Apply the rotary transformation
    rotated = torch.cat((-x2, x1), dim=-1)
    x_rotated = (x * cos) + (rotated * sin)

    return x_rotated.to(dtype=x.dtype)

In [35]:
# Settings
batch_size = 2
context_len = 5
num_heads = 4
head_dim = 16

# Instantiate RoPE parameters
cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

# Dummy query and key tensors
torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
keys = torch.randn(batch_size, num_heads, context_len, head_dim)

# Apply rotary position embeddings
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)

In [38]:
print(cos.shape)
print(sin.shape)

torch.Size([5, 16])
torch.Size([5, 16])


In [39]:
print(queries.shape)
print(keys.shape)

torch.Size([2, 4, 5, 16])
torch.Size([2, 4, 5, 16])


In [40]:
print(queries_rot.shape)
print(keys_rot.shape)

torch.Size([2, 4, 5, 16])
torch.Size([2, 4, 5, 16])


&nbsp;
## 1.5 Add RoPE to MultiHeadAttention module

- It's important to note that GPT applies the positional embeddings to the inputs, whereas **Llama applies rotations to the query and key vectors in the self-attention mechanism itself**
- Here, we modify the `MultiHeadAttention` class with the appropriate RoPE code
- In addition, we remove the `qkv_bias` option and hardcode the `bias=False` setting
- Also, we add a dtype setting to be able to instantiate the model with a lower precision later
 - Tip: since the `TransformerBlock`s (in the next section) are repeated exactly, we could simplify the code and only initialize the buffers once instead for each `MultiHeadAttention` module; however, we add the precomputed RoPE parameters to the `MultiHeadAttention` class so that it can function as a standalone module

In [41]:
#####################################
# Chapter 3
#####################################
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):  # ,dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        ################################### NEW ###################################
        # Set bias=False and dtype=dtype for all linear layers below
        ###########################################################################
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)  # Linear layer to combine head outputs
        # self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ################################### NEW ###################################
        cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_length)
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)
        ###########################################################################


    def forward(self, x):

        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        ################################### NEW ###################################
        keys = compute_rope(keys, self.cos, self.sin)
        queries = compute_rope(queries, self.cos, self.sin)
        ###########################################################################

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

In [43]:
# Settings
batch_size = 1
context_len = 100
max_context_len = 4096
embed_dim = 128
num_heads = 4


example_batch = torch.randn((batch_size, context_len, embed_dim))

mha = MultiHeadAttention(
    d_in=embed_dim,
    d_out=embed_dim,
    context_length=max_context_len,
    num_heads=num_heads
)

out = mha(example_batch)

del mha  # delete to free up memory

In [44]:
example_batch.shape

torch.Size([1, 100, 128])

In [45]:
out.shape

torch.Size([1, 100, 128])

&nbsp;
## 1.6 Update the TransformerBlock module

- At this stage, most of the hard work is already done; we can now update the `TransformerBlock` to use the code we implemented above
- This means we
 - replace LayerNorm with RMSNorm
 - remove dropout
 - remove the `qkv_bias` setting
 - add the `dtype` setting

In [46]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dtype=cfg["dtype"]  # NEW
            # dropout=cfg["drop_rate"],
            # qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)

        ################################### NEW ###################################
        # self.norm1 = LayerNorm(cfg["emb_dim"])
        # self.norm2 = LayerNorm(cfg["emb_dim"])
        self.norm1 = RMSNorm(cfg["emb_dim"])
        self.norm2 = RMSNorm(cfg["emb_dim"])
        ###########################################################################

        # self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

&nbsp;
## 1.7 Update the model class

- As you may recall from [chapter 5](../01_main-chapter-code/ch05.ipynb), the `TransformerBlock` is a repeated block within the main model
- Our Llama model is almost complete; we just have to update the model code surrounding the `TransformerBlock`
- This means we
  - remove absolute positional embeddings since we have RoPE embeddings now
  - replace LayerNorm with RMSNorm
  - remove dropout
  - add the dtype setting

In [76]:
# class GPTModel(nn.Module):
class Llama2Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])
        # self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        ################################### NEW ###################################
        # self.final_norm = LayerNorm(cfg["emb_dim"])
        self.final_norm = RMSNorm(cfg["emb_dim"])
        ###########################################################################
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

    def forward(self, in_idx):
        # batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds  # + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

&nbsp;
## 2. Initialize model

- The model code is now complete, and we are ready to initialize it
- In [chapter 5](../01_main-chapter-code/ch05.ipynb), we used the following config file to specify the 124M-parameter GPT model:

In [77]:
LLAMA2_CONFIG_7B = {
    "vocab_size": 32000,     # Vocabulary size
    "context_length": 4096,  # Context length
    "emb_dim": 4096,         # Embedding dimension
    "n_heads": 32,           # Number of attention heads
    "n_layers": 32,          # Number of layers
    "hidden_dim": 11008,     # NEW: Size of the intermediate dimension in FeedForward
    "dtype": torch.bfloat16  # NEW: Lower-precision dtype to reduce memory usage
}

- Using these settings, we can now initialize a Llama 2 7B model (note that this requires ~26 GB of memory)

In [ ]:
model = Llama2Model(LLAMA2_CONFIG_7B)

In [50]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 6,738,415,616


- As shown above, the model contains 6.7 billion parameters (commonly rounded and referred to as a 7B model)
- Additionally, we can calculate the memory requirements for this model using the code below:

In [51]:
def model_memory_size(model, input_dtype=torch.float32):
    total_params = 0
    total_grads = 0
    for param in model.parameters():
        # Calculate total number of elements per parameter
        param_size = param.numel()
        total_params += param_size
        # Check if gradients are stored for this parameter
        if param.requires_grad:
            total_grads += param_size

    # Calculate buffer size (non-parameters that require memory)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # Size in bytes = (Number of elements) * (Size of each element in bytes)
    # We assume parameters and gradients are stored in the same type as input dtype
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # Convert bytes to gigabytes
    total_memory_gb = total_memory_bytes / (1024**3)

    return total_memory_gb

print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")

float32 (PyTorch default): 52.33 GB
bfloat16: 26.17 GB


- Transfer the model to a GPU

In [52]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

model.to(device);

# 3. Tokenizer

- In this section, we are going to load the tokenizer for the model
- Llama 2 uses Google's [SentencePiece](https://github.com/google/sentencepiece) tokenizer instead of OpenAI's [Tiktoken](https://github.com/openai/tiktoken) (but Llama 3 uses Tiktoken)
- Meta AI shared the original Llama 2 model weights and tokenizer vocabulary on the Hugging Face Hub
- We will download the tokenizer vocabulary from the Hub and load it into SentencePiece
- Uncomment and run the following code to install the required libraries:

- Please note that Meta AI requires that you accept the Llama 2 licensing terms before you can download the files; to do this, you have to create a Hugging Face Hub account and visit the [meta-llama/Llama-2-7b](https://huggingface.co/meta-llama/Llama-2-7b) repository to accept the terms
- Next, you will need to create an access token; to generate an access token with READ permissions, click on the profile picture in the upper right and click on "Settings"


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/settings.webp?1" width="300px">

- Then, create and copy the access token so you can copy & paste it into the next code cell

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/access-token.webp?1" width="600px">

In [58]:
# !pip install python-dotenv

In [59]:
from dotenv import load_dotenv

In [60]:
load_dotenv()

True

In [62]:
import os
# os.environ["HF_ACCESS_TOKEN"]

In [64]:
from huggingface_hub import login
access_token = os.getenv("HF_ACCESS_TOKEN")

In [70]:
login(token=access_token)

In [71]:
from huggingface_hub import hf_hub_download

tokenizer_file = hf_hub_download(
    repo_id="meta-llama/Llama-2-7b",
    filename="tokenizer.model",
    local_dir="Llama-2-7b"
)

In [72]:
tokenizer_file

'Llama-2-7b/tokenizer.model'

In [73]:
import sentencepiece as spm


class LlamaTokenizer:
    def __init__(self, tokenizer_file):
        sp = spm.SentencePieceProcessor()
        sp.load(tokenizer_file)
        self.tokenizer = sp

    def encode(self, text):
        return self.tokenizer.encode_as_ids(text)

    def decode(self, ids):
        return self.tokenizer.decode_pieces(ids)


tokenizer = LlamaTokenizer(tokenizer_file)

In [75]:
from scripts.previous_chapters import generate, text_to_token_ids, token_ids_to_text


torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves", tokenizer).to(device),
    max_new_tokens=30,
    context_size=LLAMA2_CONFIG_7B["context_length"],
    top_k=1,
    temperature=0.
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

TypeError: LlamaTokenizer.encode() got an unexpected keyword argument 'allowed_special'

# 4. Load pretrained weights

In [ ]:
weights_file = hf_hub_download(
   repo_id="meta-llama/Llama-2-7b",
   filename="consolidated.00.pth",
   local_dir="Llama-2-7b"
)

In [ ]:
weights = torch.load(weights_file, weights_only=True)

In [ ]:
list(weights.keys())[:15]

- The following function, modeled after the `load_weights_into_gpt` function in [chapter 5](../01_main-chapter-code/ch05.ipynb), loads the pretrained weights into our Llama 2 model:

In [69]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")

    if isinstance(right, torch.Tensor):
        return torch.nn.Parameter(right.clone().detach())
    else:
        return torch.nn.Parameter(torch.tensor(right))


def load_weights_into_llama(model, param_config, params):
    model.tok_emb.weight = assign(model.tok_emb.weight, params["tok_embeddings.weight"])

    for l in range(param_config["n_layers"]):

        # Load attention weights
        model.trf_blocks[l].att.W_query.weight = assign(
            model.trf_blocks[l].att.W_query.weight,
            params[f"layers.{l}.attention.wq.weight"]
        )
        model.trf_blocks[l].att.W_key.weight = assign(
            model.trf_blocks[l].att.W_key.weight,
            params[f"layers.{l}.attention.wk.weight"]
        )
        model.trf_blocks[l].att.W_value.weight = assign(
            model.trf_blocks[l].att.W_value.weight,
            params[f"layers.{l}.attention.wv.weight"]
        )
        model.trf_blocks[l].att.out_proj.weight = assign(
            model.trf_blocks[l].att.out_proj.weight,
            params[f"layers.{l}.attention.wo.weight"]
        )
        model.trf_blocks[l].norm1.weight = assign(
            model.trf_blocks[l].norm1.weight,
            params[f"layers.{l}.attention_norm.weight"]
        )

        # Load FeedForward weights
        model.trf_blocks[l].ff.fc1.weight = assign(
            model.trf_blocks[l].ff.fc1.weight,
            params[f"layers.{l}.feed_forward.w1.weight"]
        )
        # For some reason w2 and w3 are provided in the wrong order in the weights file
        model.trf_blocks[l].ff.fc2.weight = assign(
            model.trf_blocks[l].ff.fc2.weight,
            params[f"layers.{l}.feed_forward.w3.weight"]
        )
        model.trf_blocks[l].ff.fc3.weight = assign(
            model.trf_blocks[l].ff.fc3.weight,
            params[f"layers.{l}.feed_forward.w2.weight"]
        )
        model.trf_blocks[l].norm2.weight = assign(
            model.trf_blocks[l].norm2.weight,
            params[f"layers.{l}.ffn_norm.weight"]
        )

    # Load output layer weights
    model.final_norm.weight = assign(model.final_norm.weight, params["norm.weight"])
    model.out_head.weight = assign(model.out_head.weight, params["output.weight"])

In [ ]:
load_weights_into_llama(model, LLAMA2_CONFIG_7B, weights)
model.to(device);

## Use the model

In [ ]:
torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort", tokenizer).to(device),
    max_new_tokens=25,
    context_size=LLAMA2_CONFIG_7B["context_length"],
    top_k=1,
    temperature=0.
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

# 5. Using the instruction-finetuned model

- As mentioned earlier, above we used the pretrained base model; if you want to use a model capable of following instructions, use the `"meta-llama/Llama-2-7b-chat"` model instead, as shown below

In [ ]:
del model  # to free up memory

weights_file = hf_hub_download(
   repo_id="meta-llama/Llama-2-7b-chat",
   filename="consolidated.00.pth",
   local_dir="Llama-2-7b-chat"
)

model = Llama2Model(LLAMA2_CONFIG_7B)
load_weights_into_llama(model, LLAMA2_CONFIG_7B, weights)
model.to(device);

torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("What do llamas eat?", tokenizer).to(device),
    max_new_tokens=25,
    context_size=LLAMA2_CONFIG_7B["context_length"],
    top_k=1,
    temperature=0.
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))